Crawl events (date, time, location, title, artists, works and image link) from the following link: https://www.lucernefestival.ch/en/program/summer-festival-23.

Insert data into a database (PostgreSQL), with a self-defined schema

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

/var/folders/gy/sfq3f6894rd1v__jp0ggg92c0000gn/T/ipykernel_9333/1222050232.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [200]:
URL = 'https://www.lucernefestival.ch/en/program/summer-festival-24'

In [201]:
# HTTP Request
webpage = requests.get(URL)

In [202]:
webpage

<Response [200]>

In [203]:
# Soup Object containiang all data
soup = BeautifulSoup(webpage.content, "html.parser")

In [204]:
soup


<!DOCTYPE HTML>

<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>Summer Festival 2024 | Lucerne Festival</title>
<meta content="None" name="description"/>
<meta content="MIR MEDIA - Digital Agency - www.mir.de" name="author"/>
<meta content="index, follow" name="robots"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0" name="viewport"/>
<link href="/static/css/lib/daterangepicker.css" rel="stylesheet" type="text/css"/>
<link href="https://cdn.jsdelivr.net/npm/@fancyapps/ui/dist/fancybox.css" rel="stylesheet"/>
<link href="/static/css/build/app.min.css" rel="stylesheet" type="text/css"/>
<link href="/static/images/favicons/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/static/images/favicons/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="/static/images/favicons/apple-icon-72x72.png" rel="apple-touch-icon

In [205]:
# Fetch links as List of Tag Objects
links = soup.find_all("p", attrs={'class':'event-title h3'})

In [206]:
links

[<p class="event-title h3">
 <a href="/en/program/youth-symphony-orchestra-of-ukraine-oksana-lyniv/2024">Youth Symphony Orchestra of Ukraine | Oksana Lyniv</a>
 </p>,
 <p class="event-title h3">
 <a href="/en/program/european-union-youth-orchestra-gianandrea-noseda-nicolas-altstaedt/2026">European Union Youth Orchestra | Gianandrea Noseda | Nicolas Altstaedt</a>
 </p>,
 <p class="event-title h3">
 <a href="/en/program/lucerne-festival-orchestra-riccardo-chailly/2028">Lucerne Festival Orchestra | Riccardo Chailly</a>
 </p>,
 <p class="event-title h3">
 <a href="/en/program/lucerne-festival-orchestra-klaus-makela-leif-ove-andsnes/2035">Lucerne Festival Orchestra | Klaus Mäkelä | Leif Ove Andsnes</a>
 </p>,
 <p class="event-title h3">
 <a href="/en/program/soloists-of-the-lucerne-festival-orchestra/2036">Soloists of the Lucerne Festival Orchestra</a>
 </p>,
 <p class="event-title h3">
 <a href="/en/program/soloists-of-the-lucerne-festival-contemporary-orchestra-lfco/2060">Soloists of the 

In [207]:
link = links[0].find('a').get('href')

In [208]:
link

'/en/program/youth-symphony-orchestra-of-ukraine-oksana-lyniv/2024'

In [209]:
product_list = "https://www.lucernefestival.ch" + link

In [212]:
product_list

'https://www.lucernefestival.ch/en/program/youth-symphony-orchestra-of-ukraine-oksana-lyniv/2024'

In [213]:
new_webpage = requests.get(product_list)

In [214]:
new_webpage

<Response [200]>

In [215]:
new_soup = BeautifulSoup(new_webpage.content, "html.parser")

In [206]:
# Find the <div> tag with class "program-item p"
program_div = new_soup.find("div", class_="program-item p")

# Extract the time text
if program_div:
    # Extract text from the <div> tag
    time_text = program_div.get_text(strip=True)
    print("Time:", time_text)
else:
    print("Time information not found")

Time: 19.30


In [211]:
# Find the <div> tag with id "program"
program_div = new_soup.find("section", id="program")

# Extract the program names
if program_div:
    # Find all <div> tags with class "program-item p"
    program_items = program_div.find_all("div", class_="program-item p")
    
    # Extract and format the text from each program item
    program_names = []
    for item in program_items:
        strong_tag = item.find("strong")
        if strong_tag:
            composer = strong_tag.get_text(strip=True)
            additional_info_tag = item.find("strong").find_next_sibling(string=True)  
            additional_info = additional_info_tag.strip() if additional_info_tag else ""  
            composition_tag = item.find("em")
            composition = composition_tag.get_text(strip=True) if composition_tag else "" 
            program_names.append(f"{composer} {additional_info}\n{composition}")
    
    # Join program names with newline
    programs = "\n".join(program_names)
    
    print("Program Names:")
    print(programs)
else:
    print("Program section not found")

Program Names:
Carlos Simon (*1986)
Fate Now Conquers
Benjamin Britten (1913–1976)
The Young Person’s Guide to the Orchestra
Richard Strauss (1864–1949)
Don Quixote


In [198]:
header_tag = new_soup.find("header", class_="page-header clip-path clr-pri")

# Extract the title text
if header_tag:
    # Find the <span> tag with class "subtitle" within the <header> tag
    subtitle_tag = header_tag.find("span", class_="subtitle")
    if subtitle_tag:
        title_text = subtitle_tag.text.strip()
        print("Title:", title_text)
    else:
        print("Subtitle tag not found within the header tag")
else:
    print("Header tag not found")

Title: Lucerne Festival Orchestra 3


In [192]:
# Find the <ul> tag with class "performers-list"
performers_ul = new_soup.find("ul", class_="performers-list")

# Extract the performer names
if performers_ul:
    # Find all <strong> tags within the <ul> tag
    performer_strong_tags = performers_ul.find_all("strong")
    
    # Extract the text from each <strong> tag and join them with comma
    performer_names = [strong_tag.get_text(strip=True) for strong_tag in performer_strong_tags]
    
    # Join performer names with comma
    performers = ", ".join(performer_names)
    
    print("Performers:", performers)
else:
    print("Performers list not found")

Performers: Soloists of the Lucerne Festival Orchestra


In [216]:
# Extract the venue
venue_div = new_soup.find("div", class_="cell large-6 subtitle")
venue_text = venue_div.get_text(strip=True)
# Split the text by the separator "|"
venue = venue_text.split("|")[-1].strip()
venue

'KKL Luzern, Concert Hall'

In [104]:
# Find the <time> tag with class="date-item h2"
date_tag = new_soup.find("time", class_="date-item h2")

# Extract the date text

date_text = date_tag.get_text().rstrip('.') 
date_text


'Tue 13.08'

In [224]:
# Find the figure tag with class "fullscreen-image"
figure_tag = new_soup.find('figure', class_='fullscreen-image')

# Extract the img src attribute
if figure_tag:
    img_tag = figure_tag.find('img')
    if img_tag:
        image_link = img_tag['src']
        print("Image Link:", image_link)
    else:
        print("Image tag not found")
else:
    print("Figure tag not found")

Image Link: /media/thumbnails/filer_public/7e/8c/7e8cc114-18d2-4b2e-a504-686db42a430b/youth_symphony_orchestra_of_ukraine_lyniv2_c_mutesouvenir_kaibienert.jpg__744x744_q85_crop_subject_location-1470%2C847_subsampling-2_upscale.jpg





#### Final code to loop through the links to get all the mentioned informations

In [71]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_title(soup):
    header_tag = soup.find("header", class_="page-header clip-path clr-pri")
    if header_tag:
        subtitle_tag = header_tag.find("span", class_="subtitle")
        if subtitle_tag:
            return subtitle_tag.text.strip()
    return ""
    
# Function to extract Venue
def get_venue(soup):
    try:
        venue_div = soup.find("div", class_="cell large-6 subtitle")
        venue_text = venue_div.get_text(strip=True)
        venue = venue_text.split("|")[-1].strip()
    except AttributeError:
        venue = ""
    return venue
    
# Function to extract Date
def get_date(soup):
    try:
        date_tag = soup.find("time", class_="date-item h2")
        date_text = date_tag.get_text().rstrip('.')
    except AttributeError:
        date_text = ""
    return date_text   
    
def get_time(soup):
    program_div = soup.find("div", class_="program-item p")
    if program_div:
        time_text = program_div.get_text(strip=True)
    return time_text


def get_image_link(soup):
    figure_tag = soup.find('figure', class_='fullscreen-image')
    if figure_tag:
        img_tag = figure_tag.find('img')
        if img_tag:
            return "https://www.lucernefestival.ch" + img_tag['src']
    return ""

def get_artists(soup):
    performers_ul = soup.find("ul", class_="performers-list")
    artists = []
    if performers_ul:
        performer_strong_tags = performers_ul.find_all("strong")
        for strong_tag in performer_strong_tags:
            artists.append(strong_tag.get_text(strip=True))
    return artists

def get_programs(soup):
    program_div = soup.find("section", id="program")
    programs = []
    if program_div:
        program_items = program_div.find_all("div", class_="program-item p")
        for item in program_items:
            strong_tag = item.find("strong")
            if strong_tag:
                composer = strong_tag.get_text(strip=True)
                additional_info_tag = item.find("strong").find_next_sibling(string=True)
                additional_info = additional_info_tag.strip() if additional_info_tag else ""
                composition_tag = item.find("em")
                composition = composition_tag.get_text(strip=True) if composition_tag else ""
                programs.append(f"{composer} {additional_info}\n{composition}")
    return programs

if __name__ == '__main__':
    URL = 'https://www.lucernefestival.ch/en/program/summer-festival-24'
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.content, "html.parser")
    links = soup.find_all("p", attrs={'class': 'event-title h3'})

    data = {"title": [], "venue": [], "date": [], "time": [], "artist": [], "program": [], "image_link": []}

    for link in links:
        event_link = link.find('a').get('href')
        event_url = "https://www.lucernefestival.ch" + event_link
        event_webpage = requests.get(event_url)
        event_soup = BeautifulSoup(event_webpage.content, "html.parser")

        title = get_title(event_soup)
        venue = get_venue(event_soup)
        date = get_date(event_soup)
        time = get_time(event_soup)
        image_link = get_image_link(event_soup)
        artists = get_artists(event_soup)
        programs = get_programs(event_soup)

        for artist in artists:
            for program in programs:
                data["title"].append(title)
                data["venue"].append(venue)
                data["date"].append(date)
                data["time"].append(time)
                data["artist"].append(artist)
                data["program"].append(program)
                data["image_link"].append(image_link)

    lucerne_df = pd.DataFrame(data)
    lucerne_df.to_csv("lucerne_festival_data.csv", header=True, index=False)


In [125]:
test_dataframe = lucerne_df.copy(deep = True)

In [85]:
lucerne_df

,title,venue,date,time,artist,program,image_link
0,Youth Symphony Orchestra of Ukraine,"KKL Luzern, Concert Hall",Tue 13.08,19.30,Youth Symphony Orchestra of Ukraine,Claude Debussy (1862–1918)\nPrélude à l'après-...,https://www.lucernefestival.ch/media/thumbnail...
1,Youth Symphony Orchestra of Ukraine,"KKL Luzern, Concert Hall",Tue 13.08,19.30,Youth Symphony Orchestra of Ukraine,Edward Elgar (1857–1934)\n,https://www.lucernefestival.ch/media/thumbnail...
2,Youth Symphony Orchestra of Ukraine,"KKL Luzern, Concert Hall",Tue 13.08,19.30,Youth Symphony Orchestra of Ukraine,Borys Ljatoschynskyj (1895–1968)\nSlavic Suite,https://www.lucernefestival.ch/media/thumbnail...
3,Youth Symphony Orchestra of Ukraine,"KKL Luzern, Concert Hall",Tue 13.08,19.30,Youth Symphony Orchestra of Ukraine,Ottorino Respighi (1879–1936)\nPini di Roma,https://www.lucernefestival.ch/media/thumbnail...
4,Youth Symphony Orchestra of Ukraine,"KKL Luzern, Concert Hall",Tue 13.08,19.30,Oksana Lyniv,Claude Debussy (1862–1918)\nPrélude à l'après-...,https://www.lucernefestival.ch/media/thumbnail...
...,...,...,...,...,...,...,...
281,Gurre-Lieder,"KKL Luzern, Concert Hall",Sun 15.09,18.30,Christina Nilsson,Arnold Schoenberg (1874–1951)\nGurre-Lieder,https://www.lucernefestival.ch/media/thumbnail...
282,Gurre-Lieder,"KKL Luzern, Concert Hall",Sun 15.09,18.30,Jamie Barton,Arnold Schoenberg (1874–1951)\nGurre-Lieder,https://www.lucernefestival.ch/media/thumbnail...
283,Gurre-Lieder,"KKL Luzern, Concert Hall",Sun 15.09,18.30,Michael Schade,Arnold Schoenberg (1874–1951)\nGurre-Lieder,https://www.lucernefestival.ch/media/thumbnail...
284,Gurre-Lieder,"KKL Luzern, Concert Hall",Sun 15.09,18.30,Michael Nagy,Arnold Schoenberg (1874–1951)\nGurre-Lieder,https://www.lucernefestival.ch/media/thumbnail...


In [90]:
# Separate dataframes for title, venue, artist, and program
title_df = lucerne_df[['title']]
venue_df = lucerne_df[['venue']]
artist_df = lucerne_df[['artist']]
program_df = lucerne_df[['program']]

#### Defining Schema for Postgres insertion

In [122]:
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values
import pandas as pd

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(user="postgres",
                                      password="postgres",
                                      host="localhost",
                                      port="5432",
                                      database="docker")

cursor = conn.cursor()

# Create tables
create_tables_query = """

-- Create venues table
CREATE TABLE IF NOT EXISTS venues (
    venue_id SERIAL PRIMARY KEY,
    venue_name VARCHAR(255)
);

-- Create artists table
CREATE TABLE IF NOT EXISTS artists (
    artist_id SERIAL PRIMARY KEY,
    artist_name VARCHAR(255)
);

-- Create programs table
CREATE TABLE IF NOT EXISTS programs (
    program_id SERIAL PRIMARY KEY,
    program_name VARCHAR(255)
);

-- Create programs table
CREATE TABLE IF NOT EXISTS titles (
    title_id SERIAL PRIMARY KEY,
    title_name VARCHAR(255)
);

-- Create events table
CREATE TABLE IF NOT EXISTS events (
    title_id INTEGER,
    venue_id INTEGER,
    event_date DATE,
    event_time TIME,
    artist_id INTEGER,
    program_id INTEGER,
    image_link TEXT
);

-- Add foreign key constraint for artist_id in events table
ALTER TABLE events ADD CONSTRAINT fk_artist_id FOREIGN KEY (artist_id) REFERENCES artists(artist_id);

-- Add foreign key constraint for venue_id in events table
ALTER TABLE events ADD CONSTRAINT fk_venue_id FOREIGN KEY (venue_id) REFERENCES venues(venue_id);

-- Add foreign key constraint for program_id in events table
ALTER TABLE events ADD CONSTRAINT fk_program_id FOREIGN KEY (program_id) REFERENCES programs(program_id);

-- Add foreign key constraint for program_id in events table
ALTER TABLE events ADD CONSTRAINT fk_title_id FOREIGN KEY (title_id) REFERENCES titles(title_id);

"""

# Execute the query to create tables
cursor.execute(create_tables_query)

# Commit the transaction
conn.commit()

In [123]:
# Iterate through the program_df dataframe to extract unique program names
unique_programs = set(program_df['program'])
unique_artist = set(artist_df['artist'])
unique_venue = set(venue_df['venue'])
unique_title = set(title_df['title'])

# Insert each unique program name into the programs table
for program_name in unique_programs:
    # Execute the SQL query to insert the program name
    cursor.execute("INSERT INTO programs (program_name) VALUES (%s) ON CONFLICT DO NOTHING", (program_name,))

for artist_name in unique_artist:
    # Execute the SQL query to insert the program name
    cursor.execute("INSERT INTO artists (artist_name) VALUES (%s) ON CONFLICT DO NOTHING", (artist_name,))

for venue_name in unique_venue:
    # Execute the SQL query to insert the program name
    cursor.execute("INSERT INTO venues (venue_name) VALUES (%s) ON CONFLICT DO NOTHING", (venue_name,))

for title_name in unique_title:
    # Execute the SQL query to insert the program name
    cursor.execute("INSERT INTO titles (title_name) VALUES (%s) ON CONFLICT DO NOTHING", (title_name,))
    
# Commit the transaction and close the cursor and connection
conn.commit()

In [136]:
# Define a function to get the ID for a given name from the database
def get_id(table_name, column_name1, column_name2, value):
    query = sql.SQL("SELECT {} FROM {} WHERE {} = %s").format(sql.Identifier(column_name1), sql.Identifier(table_name), sql.Identifier(column_name2))
    cursor.execute(query, (value,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        return None

# Iterate through each row in the DataFrame and replace the column values with their corresponding IDs
for index, row in test_dataframe.iterrows():
    title_id = get_id('titles', 'title_id', 'title_name', row['title'])
    venue_id = get_id('venues', 'venue_id', 'venue_name', row['venue'])
    artist_id = get_id('artists', 'artist_id', 'artist_name', row['artist'])
    program_id = get_id('programs', 'program_id', 'program_name', row['program'])
    
    # Replace the column values with their corresponding IDs
    test_dataframe.at[index, 'title'] = title_id
    test_dataframe.at[index, 'venue'] = venue_id
    test_dataframe.at[index, 'artist'] = artist_id
    test_dataframe.at[index, 'program'] = program_id

Found ID 39 for Youth Symphony Orchestra of Ukraine
Found ID 1 for KKL Luzern, Concert Hall
Found ID 44 for Youth Symphony Orchestra of Ukraine
Found ID 84 for Claude Debussy (1862–1918)
Prélude à l'après-midi d'un faune
Found ID 39 for Youth Symphony Orchestra of Ukraine
Found ID 1 for KKL Luzern, Concert Hall
Found ID 44 for Youth Symphony Orchestra of Ukraine
Found ID 27 for Edward Elgar (1857–1934)

Found ID 39 for Youth Symphony Orchestra of Ukraine
Found ID 1 for KKL Luzern, Concert Hall
Found ID 44 for Youth Symphony Orchestra of Ukraine
Found ID 59 for Borys Ljatoschynskyj (1895–1968)
Slavic Suite
Found ID 39 for Youth Symphony Orchestra of Ukraine
Found ID 1 for KKL Luzern, Concert Hall
Found ID 44 for Youth Symphony Orchestra of Ukraine
Found ID 47 for Ottorino Respighi (1879–1936)
Pini di Roma
Found ID 39 for Youth Symphony Orchestra of Ukraine
Found ID 1 for KKL Luzern, Concert Hall
Found ID 8 for Oksana Lyniv
Found ID 84 for Claude Debussy (1862–1918)
Prélude à l'après-mid

In [137]:
test_dataframe

,title,venue,date,time,artist,program,image_link
0,39,1,Tue 13.08,19.30,44,84,https://www.lucernefestival.ch/media/thumbnail...
1,39,1,Tue 13.08,19.30,44,27,https://www.lucernefestival.ch/media/thumbnail...
2,39,1,Tue 13.08,19.30,44,59,https://www.lucernefestival.ch/media/thumbnail...
3,39,1,Tue 13.08,19.30,44,47,https://www.lucernefestival.ch/media/thumbnail...
4,39,1,Tue 13.08,19.30,8,84,https://www.lucernefestival.ch/media/thumbnail...
...,...,...,...,...,...,...,...
281,42,1,Sun 15.09,18.30,51,67,https://www.lucernefestival.ch/media/thumbnail...
282,42,1,Sun 15.09,18.30,61,67,https://www.lucernefestival.ch/media/thumbnail...
283,42,1,Sun 15.09,18.30,71,67,https://www.lucernefestival.ch/media/thumbnail...
284,42,1,Sun 15.09,18.30,12,67,https://www.lucernefestival.ch/media/thumbnail...


In [159]:
# Convert the event date strings to the proper format
test_dataframe['date'] = pd.to_datetime(test_dataframe['date'], format='%a %d.%m').dt.strftime('%Y-%m-%d')


In [161]:
test_dataframe.drop(['event_date'], axis = 1, inplace = True) 

In [162]:
# Convert the 'event_date' column to datetime format
test_dataframe['date'] = pd.to_datetime(test_dataframe['date'])

# Replace the year component in the 'event_date' column
test_dataframe['date'] = test_dataframe['date'].apply(lambda x: x.replace(year=2024).strftime('%Y-%m-%d'))

test_dataframe

,title,venue,date,time,artist,program,image_link
0,39,1,2024-08-13,19.30,44,84,https://www.lucernefestival.ch/media/thumbnail...
1,39,1,2024-08-13,19.30,44,27,https://www.lucernefestival.ch/media/thumbnail...
2,39,1,2024-08-13,19.30,44,59,https://www.lucernefestival.ch/media/thumbnail...
3,39,1,2024-08-13,19.30,44,47,https://www.lucernefestival.ch/media/thumbnail...
4,39,1,2024-08-13,19.30,8,84,https://www.lucernefestival.ch/media/thumbnail...
...,...,...,...,...,...,...,...
281,42,1,2024-09-15,18.30,51,67,https://www.lucernefestival.ch/media/thumbnail...
282,42,1,2024-09-15,18.30,61,67,https://www.lucernefestival.ch/media/thumbnail...
283,42,1,2024-09-15,18.30,71,67,https://www.lucernefestival.ch/media/thumbnail...
284,42,1,2024-09-15,18.30,12,67,https://www.lucernefestival.ch/media/thumbnail...


In [192]:
# Convert 'artist' and 'program' columns to integer type
test_dataframe['artist'] = test_dataframe['artist'].astype(int)
test_dataframe['program'] = test_dataframe['program'].astype(int)


In [193]:
type(test_dataframe['artist'])

pandas.core.series.Series

In [182]:
# Convert the 'date' column to datetime format with specified format
test_dataframe['date'] = pd.to_datetime(test_dataframe['date'], format='%Y-%m-%d')

In [185]:
# Check for missing or null values in the 'date' column
missing_values = test_dataframe['date'].isnull().sum()
print("Number of missing values in 'date' column:", missing_values)


print(test_dataframe['date'].head())

# Check the unique values in the 'date' column
unique_dates = test_dataframe['date'].unique()
print("Unique dates:", unique_dates)


Number of missing values in 'date' column: 0
0   2024-08-13
1   2024-08-13
2   2024-08-13
3   2024-08-13
4   2024-08-13
Name: date, dtype: datetime64[ns]
Unique dates: <DatetimeArray>
['2024-08-13 00:00:00', '2024-08-14 00:00:00', '2024-08-16 00:00:00',
 '2024-08-17 00:00:00', '2024-08-18 00:00:00', '2024-08-19 00:00:00',
 '2024-08-20 00:00:00', '2024-08-21 00:00:00', '2024-08-22 00:00:00',
 '2024-08-23 00:00:00', '2024-08-24 00:00:00', '2024-08-25 00:00:00',
 '2024-08-27 00:00:00', '2024-08-28 00:00:00', '2024-08-29 00:00:00',
 '2024-08-30 00:00:00', '2024-08-31 00:00:00', '2024-09-01 00:00:00',
 '2024-09-02 00:00:00', '2024-09-03 00:00:00', '2024-09-04 00:00:00',
 '2024-09-05 00:00:00', '2024-09-06 00:00:00', '2024-09-07 00:00:00',
 '2024-09-08 00:00:00', '2024-09-09 00:00:00', '2024-09-10 00:00:00',
 '2024-09-11 00:00:00', '2024-09-12 00:00:00', '2024-09-13 00:00:00',
 '2024-09-14 00:00:00', '2024-09-15 00:00:00']
Length: 32, dtype: datetime64[ns]


In [189]:
# Convert the time format in the 'event_time' column
test_dataframe['time'] = test_dataframe['time'].apply(lambda x: f'{int(x.split(".")[0]):02d}:{int(x.split(".")[1]):02d}:00')

In [190]:
test_dataframe

,title,venue,date,time,artist,program,image_link
0,39,1,2024-08-13,19:30:00,44,84,https://www.lucernefestival.ch/media/thumbnail...
1,39,1,2024-08-13,19:30:00,44,27,https://www.lucernefestival.ch/media/thumbnail...
2,39,1,2024-08-13,19:30:00,44,59,https://www.lucernefestival.ch/media/thumbnail...
3,39,1,2024-08-13,19:30:00,44,47,https://www.lucernefestival.ch/media/thumbnail...
4,39,1,2024-08-13,19:30:00,8,84,https://www.lucernefestival.ch/media/thumbnail...
...,...,...,...,...,...,...,...
281,42,1,2024-09-15,18:30:00,51,67,https://www.lucernefestival.ch/media/thumbnail...
282,42,1,2024-09-15,18:30:00,61,67,https://www.lucernefestival.ch/media/thumbnail...
283,42,1,2024-09-15,18:30:00,71,67,https://www.lucernefestival.ch/media/thumbnail...
284,42,1,2024-09-15,18:30:00,12,67,https://www.lucernefestival.ch/media/thumbnail...


In [ ]:
# Convert the DataFrame to a list of tuples
data = [tuple(row) for row in test_dataframe.values]

# Define the insert query
insert_query = """
    INSERT INTO events (title_id, venue_id, event_date, event_time, artist_id, program_id,  image_link)
    VALUES %s
"""

# Execute the insert query with the data
execute_values(cursor, insert_query, data)

# Commit the transaction
conn.commit()
